# &#x1F380; Decorators

Based on the PyCon 2024 &#x1F40D; tutorial "Reuven M. Lerner: All about decorators" 

&#x1F3AC; https://www.youtube.com/watch?v=THv-m8niDho

&#x1F47E; https://github.com/reuven/2024-pycon-decorators/

## &#x1F4D6; Contents  

1. [What are decorators?](#what_dec) <br>
1.1 [DRY rule](#dry) <br>
1.2 [Use cases](#use_cases)
2.  [How to write a decorator](#write_dec) <br>
2.1 [Exercise: Shouter](#shouter) <br>
2.2 [Exercise: Timing of functions](#timefunc) <br>
3.  [Outer function storage](#outer) <br>
3.1 [Memoization](#memo) <br>
3.2 [Exercise: Once per minute](#once_min) <br>
4. [Modifying inputs and outputs](#mod) <br>
4.1 [Exercise: Only ints](#only_ints) <br>
4.2 [Functools](#functools)
5. [Decorators with arguments](#deco_args) <br>
5.1 [Exercise: Only these types](#only_types) <br>
5.2 [Stacking decorators](#stack_decs) <br>
6. [Decorating classes](#dec_class) <br>

# &#x1F4A1;   What are decorators? <a class="anchor" id="what_dec"></a>

Decorators are best introduced via a story &#x1F3F0; &#x1F4D4; 

Once a upon a time, we had two basic functions...

In [1]:
def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(a())
print(b())

A!

B!



One day, you receive a missive &#x1F4DC; from the king &#x1F451; : 

> Everything we print in this noble kingdom must have dashed lines above and below it. Please execute this request forthwith.

&#x1F643;...But you are an experienced programmer.

In [1]:
lines = '-' * 60 +'\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



Problem solved?! Not quite - the king has plans to expand his kingdom &#x1F3F0; and is sceptical of how this solution will scale.

# &#x1F3DC;&#xFE0F; Don't Repeat Yourself (DRY) rule of programming <a class="anchor" id="dry"></a>

Currently, we have a technique that requires us to include `lines` in *every* function we write -- the epitome of repetition!

So, how can I include `lines` without explicitly including `lines` in every function?

> Use [Python tutor](https://pythontutor.com/) for visualisations &#x1F4F9;


In [2]:
# Option 1: write a new function that takes a function as an argument

lines = '-' * 40 +'\n'

def with_lines(func):
    return f'{lines}{func()}{lines}' # passes a function as an argument into the return str 

def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(with_lines(a))
print(with_lines(b))

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



This solution works because:

> Functions are first class &#x1F947; objects in Python &#x1F40D; . This means we can consider them as **verbs** and **nouns**, and can store them in variables.

This solution scales but observe that `with_lines` is executing `a()` and `b()` -- not us. We're passing `a()` and `b()` to `with_lines`.

So, now our API is broken and the peasants are threatening to revolt &#x1F525; let's try again...

In [3]:
# Option 2: turn with_lines into a function that returns a function
# i.e. create a closure!

lines = '-' * 40 +'\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper 

def a():
    return f'A!\n'

with_lines_a = with_lines(a)

def b():
    return f'B!\n'

with_lines_b = with_lines(b)

print(with_lines_a())
print(with_lines_b())

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



What is happening here?

1. We define `wrapper` only when we run `with_lines`. So, we call `with_lines`, define `wrapper` and `wrapper` is a local variable.

2. But `wrapper` doesn't execute at this point -- we only *return* it. Result: calling `with_lines` returns a function `wrapper`.

3. Now, `wrapper` is going to return the result of executing a function `(func)` with lines around it.

> The LEGB rule allows the inner &#x1F381; function (`wrapper`) access to the outer &#x1F30C; function's `with_lines` local variables but not vice versa &#x262F;

This is progress but the API is still broken, and the second-hand pitchfork market is booming...&#x1F525; &#129683;

In [4]:
# Option 3: don't assign to with_lines_a and with_lines_b
# rather, assign to a and b

lines = '-' * 40 +'\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper 

def a():           # defining a()...
    return f'A!\n'

a = with_lines(a)  # ...only to redefine it? It's like wrapping (or decorating) a -- *lightbulb moment*

def b():
    return f'B!\n'

b = with_lines(b)

print(a())
print(b())

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



Success! We've preserved the original API and narrowly avoided a revolt! &#x1F605; &#x1F525; &#129683;  

But this solution is ugly. You observe the decorative hangings of the king's chamber and have an idea &#x1F4A1; ...

In [5]:
# Option 4: use Python decorator syntax

lines = '-' * 40 +'\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper 

@with_lines    # this is equivalent to line 14 -- decorator syntax
def a():
    return f'A!\n'

# a = with_lines(a)

@with_lines
def b():
    return f'B!\n'

# b = with_lines(b)

print(a())
print(b())

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



Finally! The king approves of this new "decorator syntax" &#x1F380; and the solution proves a success. In reward for your ingenuity &#129504;, his majesty offers an additional ration of wheat &#x1F33E; 

The kingdom &#x1F3F0; lives happily ever after in lined glory &#x1F4B8; &#x1F451; 

> The End &#x1F4D4;

# &#128736;&#65039; Use cases <a class="anchor" id="use_cases"></a>

1. **Multiple functions that do the same thing -- I can extract this functionality into a decorator and thus DRY up my code** &#x1F3DC;&#xFE0F;

2. Timing functions

3. Security of functions

4. Setting values before/after (setup/teardown)

5. Logging

6. Checking/filtering inputs

7. Filtering outputs

```python
# let's use @with_lines to decorate another function

@with_lines
def add(x, y):
    return f'{x + y}\n'

print(add(4, 6))
```

Why the error? &#x274C; 

> Because we're not calling `add` -- we're calling `wrapper`. And `wrapper` takes 0 positional arguments.

So we must rewrite `wrapper` &#x1F381;

In [6]:
def with_lines(func):
    def wrapper(*args):    # take any number of positional arguments + creates a tuple "args"
        return f'{lines}{func(*args)}{lines}'    # turn args into arguments to func() 
    return wrapper 

@with_lines
def add(x, y):
    return f'{x + y}\n'

print(add(4, 6))

----------------------------------------
10
----------------------------------------



# &#x1F380; How to write a decorator <a class="anchor" id="write_dec"></a>

1. The outer function, the decorator, takes one argument, a function `func()` -- this is the function that will be decorated. This outer function is invoked once when we define the decorated function.

2. The inner function, typically called `wrapper`, takes `*args` (and maybe `**kwargs`). It is invoked every time the original function is invoked. It can do whatever it wants with the arguments and outputs.

3. After defining the inner function, the outer function returns `wrapper`.

4. We can then decorate any function with `@deco_name`

# &#x1F4DD; Exercise: `shouter` <a class="anchor" id="shouter"></a>

1. Write a decorator, `shouter`, that decorates functions that return strings.
2. Any such function's ouput will return all `IN CAPS` and with an exclamation point at the end.

Example:

``` Python
@shouter
def hello(name):
    return f'Hello, {name}'
    
print(hello('Zoe'))    # output HELLO, ZOE!

```

Hint: use `str.upper()`

In [7]:
def shouter(func):    # outer function / decorator function
     def wrapper(*args):    # inner function -- runs each time we invoke hello 
        return f'{func(*args).upper()}!'    
     return wrapper    # return wrapper object -- don't wrapper()

@shouter
def hello(name):
    return f'Hello, {name}'
    
print(hello('Zoe')) 

HELLO, ZOE!


# &#x1F4DD; Exercise: Timing of functions <a class="anchor" id="timefunc"></a>

Write a decorator, `timefunc`, that won't change the inputs or outputs of the decorated function. However, it will keep track of how long it took to run the function and will write it's results to a file, `timing.txt`.

Every time I run the decorated function, I'll get another line in `timing.txt` containing: the function name, when it was started and how long it took.

Hints:

1. Normally, writing to a file with `w` will overwrite the file's contents. You should open a file with `a` append to avoid this.

2. You can get the function name from the `__name__` attribute
3. You can get the current Unix time (seconds since 1 Jan 1970) with `time.time()`

```python
import random
import time

@timefunc
def slow_add(a, b):
    time.sleep(random.randint(3))
    return a + b

@timefunc
def slow_mul(a, b):
    time.sleep(random.randint(3))
    return a * b

print(slow_add(2, 3))
print(slow_mul(4, 5))
```

In [8]:
import random
import time

def timefunc(func):
    def wrapper(*args):
        start_time = time.time()
        value = func(*args)
        end_time = time.time()
        
        with open('timing.txt', 'a') as f:
            f.write(f'{func.__name__}\t{start_time}\t{end_time - start_time}\n')
        return value   
    return wrapper 
    
    
@timefunc
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b

@timefunc
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b

print(slow_add(2, 3))
print(slow_mul(4, 5))

5
20


In [9]:
!type timing.txt

slow_add	1726044724.6541774	3.0010738372802734
slow_mul	1726044727.657731	2.0015861988067627
slow_add	1726044743.8495271	0.0
slow_mul	1726044743.8505309	2.0016212463378906
slow_add	1726125079.7179966	3.0004806518554688
slow_mul	1726125082.7195177	3.001563787460327
slow_add	1726296556.911066	0.0
slow_mul	1726296556.9135127	2.000694751739502
slow_add	1726382509.9388344	2.000650405883789
slow_mul	1726382511.9404812	3.00097393989563
slow_add	1726382906.84629	2.0007424354553223
slow_mul	1726382908.8479533	2.0007598400115967
slow_add	1726835744.6025093	1.0011749267578125
slow_mul	1726835745.6246881	0.0


> We could improve this by adding an asynchronous tool such as Celery 🥬🌿

# &#x1F30C; Outer function storage <a class="anchor" id="outer"></a>

As we've seen, the inner function has access to the outer function's parameters and local variables. We can extend this idea -- to **store** data in the outer function. This data persists across calls to our inner (wrapper) function.

In [10]:
# I want to know how many times a function has been invoked
# and print this to the screen every time we call it

# anti-solution 

def count_calls(func):
    def wrapper(*args):
        counter = 0
        value = func(*args)
        counter += 1
        print(f'{func.__name__}: {counter=}')
        return value   
    return wrapper 
    
    
@count_calls
def add(a, b):
    return a + b

@count_calls
def mul(a, b):
    return a * b

print(add(2, 3))
print(mul(4, 5))
print(add(2, 3))
print(mul(4, 5))
print(add(2, 3))
print(mul(4, 5))

add: counter=1
5
mul: counter=1
20
add: counter=1
5
mul: counter=1
20
add: counter=1
5
mul: counter=1
20


Why doesn't this work? &#x274C; Because

> `counter` is reset to 0 every time `wrapper` is called


```python
# anti-solution 2

def count_calls(func):
    counter = 0     
    def wrapper(*args):
        value = func(*args)
        counter += 1
        print(f'{func.__name__}: {counter=}')
        return value   
    return wrapper 
    
    
@count_calls
def add(a, b):
    return a + b

@count_calls
def mul(a, b):
    return a * b

print(add(2, 3))
print(mul(4, 5))
print(add(2, 3))
print(mul(4, 5))
print(add(2, 3))
print(mul(4, 5))
```

Why the error? &#x274C; Because

> Assignment of variables within a function is automatically local &#x1F3E0; So, when we define `wrapper`, it sees 
> ```python
counter = counter + 1
>```
but can't find a the variable `counter` for the rhs

In [11]:
# solution

def count_calls(func):
    counter = 0   
    def wrapper(*args):
        nonlocal counter    
        value = func(*args)
        counter += 1
        print(f'{func.__name__}: {counter=}')
        return value   
    return wrapper 
    
    
@count_calls
def add(a, b):
    return a + b

@count_calls
def mul(a, b):
    return a * b

print(add(2, 3))
print(mul(4, 5))
print(add(2, 3))
print(mul(4, 5))
print(add(2, 3))
print(mul(4, 5))

add: counter=1
5
mul: counter=1
20
add: counter=2
5
mul: counter=2
20
add: counter=3
5
mul: counter=3
20


# &#x1F4BE; Memoization <a class="anchor" id="memo"></a>

This is an old technique for caching. The idea is that every time we call a function, we want to check the arguments passed to the function:

- If the arguments **haven't** been seen before, we run the function and cache the results.
- Else grab the results from the cache and return them.

In [12]:
# how to memoize

def memoize(func):
    cache = {}
    def wrapper(*args):
        if args not in cache:
            print(f'\tActually calling {func.__name__} with {args}')
            cache[args] = func(*args)    
        return cache[args]
    return wrapper

@memoize
def add(a, b):
    return a + b

@memoize
def mul(a, b):
    return a * b

print(add(2, 3))
print(mul(4, 5))
print(add(2, 3))
print(mul(4, 5))
print(add(0, 8))
print(mul(2, 3))

	Actually calling add with (2, 3)
5
	Actually calling mul with (4, 5)
20
5
20
	Actually calling add with (0, 8)
8
	Actually calling mul with (2, 3)
6


# &#x1F4DD; Exercise: `once_per_minute` <a class="anchor" id="once_min"></a>

Write a decorator, `once_per_minute`, which ensures a function can only run **once** in any 60-second period. Every time the function starts, it checks to ensure that at least 60 seconds have passed.

If 60 seconds haven't passed since the last run, we should raise an exception.

Else, reset the counter to the most recent start time.

Hint: use `time.time()` to get the current Unix time (number of seconds since 1 Jan 1970)

In [13]:
class CalledTooOftenError(Exception):
    pass


def once_per_minute(func):  
    last_ran_at = 0
    def wrapper(*args):
        nonlocal last_ran_at
        current_time = time.time()
        
        if current_time - last_ran_at < 60:    # need nonlocal bcoz of this reading
            raise CalledTooOftenError('Too soon!')
        last_ran_at = current_time
        
        value = func(*args)
        return value   
    return wrapper 


@once_per_minute
def add(a, b):
    return a + b

@once_per_minute
def mul(a, b):
    return a * b

print(add(2, 3))
print(mul(4, 5))
print(add(6, 7))
print(mul(1, 8))

5
20


CalledTooOftenError: Too soon!

# &#x1F527; Modifying inputs & outputs <a class="anchor" id="mod"></a>

When we write a decorator, we have two opportunities to do stuff:

1. When we define the function and it's originally decorated. This is done by the outer function.

2. When we run the function, meaning that the inner (`wrapper`) function is invoked.

This means that `wrapper` 🎁 gets the arguments we wanted to pass to the original function, and then invokes (or not) the original function. 

We can use this opportunity to filter or modify the arguments passed, as well as any results returned.

In [14]:
# Let's write a decorator that takes all of the arguments passed to
# a function, and assumes they are all integers. It only passes
# along those integers that are odd.

def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total

# normally, I could do this:
mysum(1, 2, 3, 4)

10

In [15]:
# but with my only_odds in place, we'll get 4 (i.e. 1 + 3)

def only_odds(func):
    def wrapper(*args):
        
        odd_numbers = []
        for one_item in args:
            if one_item % 2 == 1:
                odd_numbers.append(one_item)
                
        value = func(*odd_numbers)
        return value 
    return wrapper
    
@only_odds
def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total


mysum(1, 2, 3, 4)

4

Great! But observe that 

> We have an iterable (`args`), we want an iterable (`args` minus even nums) and we can define a mapping &#x1F30D; between them

Enter our old friend comprehensions!

In [16]:
# Let's tighten up the code

def only_odds(func):
    def wrapper(*args):
        
        # list comprehension!
        odd_numbers = [one_arg
                       for one_arg in args
                       if one_arg % 2 == 1]
       
        value = func(*odd_numbers)
        return value 
    return wrapper
    
@only_odds
def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total


mysum(1, 2, 3, 4)

4

In [17]:
# Keep tightening

def only_odds(func):
    def wrapper(*args):
       
        # job security!
        value = func(*[one_arg
                      for one_arg in args
                      if one_arg % 2 == 1])
        return value 
    return wrapper
    
@only_odds
def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total


mysum(1, 2, 3, 4)

4

Why does this work? &#x2705;

> We create an anonymous list and the `*` removes the parentheses. Then pass the individual args from the list to `func()`

# &#x1F4DD; Exercise: `only_ints` <a class="anchor" id="only_ints"></a>

Write a decorator that ensures that only integers will be passed to the underlying function.

Hint: Don't use `type(x) == int` in your code. Rather, use `isinstance(x, int)` to check. This is because `isinstance` checks subclasses as well. 

Example:

I should be able to call

```python
mysum(1, 2, 'zoe', 3, 4, 'python')
```

and it'll work fine, returning 10 and not raising an exception.

In [18]:
def only_ints(func):
    def wrapper(*args):
        
        value = func(*[one_arg
                      for one_arg in args
                      if isinstance(one_arg, int)])
        return value 
    return wrapper
    
@only_ints
def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total


mysum(1, 2, 'zoe', 3, 4, 'python')

10

# &#128736;&#65039; `functools` <a class="anchor" id="functools"></a>

In [19]:
# suppose we want info on mysum

help(mysum)

Help on function wrapper in module __main__:

wrapper(*args)



 This is true but uninformative &#x1F532;
 
 > Use a decorator `@functools` on `wrapper`

In [20]:
import functools

def only_ints(func):
    
    @functools.wraps(func)
    def wrapper(*args):
        
        value = func(*[one_arg
                      for one_arg in args
                      if isinstance(one_arg, int)])
        return value 
    return wrapper
    
@only_ints
def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total


mysum(1, 2, 'zoe', 3, 4, 'python')

10

In [21]:
help(mysum)

Help on function mysum in module __main__:

mysum(*numbers)



# &#x1F380; Decorators with arguments <a class="anchor" id="deco_args"></a>

What if I want my decorator to take arguments? 

Earlier, we wrote a decorator `timefunc` that times ⏲️ the running of a function. I asked you to write the output into a file called `timing.txt`. Wouldn't it be convenient if we could configure this when we invoke our function?

In order to have a decorator take an argument, we need three &#x0033;&#xFE0F;&#x20E3; levels of functions:

1. Outer function is our decorator, and now takes the argument that the user passes in the `@` expression
2. The middle function takes `func` as an argument -- it is passed the function that we are decorating 
3. The inner function, `wrapper`, remains as before.

You can think of this as breaking the outer function in two &#x0032;&#xFE0F;&#x20E3; -- one piece takes the argument, the other the function.

In [22]:
import time
import random 

def timefunc(filename):
    def middle(func):
        def wrapper(*args):
            start_time = time.time()
            value = func(*args)
            end_time = time.time()

            with open(filename, 'a') as f:
                f.write(f'{func.__name__}\t{start_time}\t{end_time - start_time}\n')
            return value   
        return wrapper 
    return middle
    
    
@timefunc('timing2.txt')
def slow_add(a, b):
    time.sleep(random.randint(0, 3))
    return a + b
# slow_add = timefunc('timing2.txt')(slow_add)

@timefunc('timing3.txt')
def slow_mul(a, b):
    time.sleep(random.randint(0, 3))
    return a * b
# slow_mul = timefunc('timing2.txt')(slow_mul)

print(slow_add(2, 3))
print(slow_mul(4, 5))

5
20


What is happening here?

> ```python 
> #slow_add = timefunc('timing2.txt')(slow_add)
> ```
> We're calling `timefunc` with an argument &#x23E9; returns a function &#x23E9; function applied to `slow_add` &#x23E9; returns a function which is assigned to `slow_add`

In [23]:
!type timing2.txt

slow_add	1726125125.0964813	1.0008561611175537
slow_add	1726126059.358613	1.000840663909912
slow_add	1726126131.0112517	1.000533103942871
slow_add	1726304726.4080563	1.001633882522583
slow_add	1726382541.1983454	0.0007691383361816406
slow_add	1726382924.1818602	2.000514030456543
slow_add	1726387544.5399296	1.0057783126831055
slow_add	1726836030.0403647	1.0008609294891357


In [24]:
!type timing3.txt

slow_mul	1726125126.0983803	2.001152276992798
slow_mul	1726126060.3594537	3.002119541168213
slow_mul	1726126132.0130918	2.001307964324951
slow_mul	1726304727.411745	0.0
slow_mul	1726382541.2011087	1.0035631656646729
slow_mul	1726382926.1823742	0.0
slow_mul	1726836031.0430522	0.0


> This is how decorators in pytest 🧪, Flask &#x2615;, Django etc. take args

# &#x1F4DD; Exercise: `only_these_types` <a class="anchor" id="only_types"></a>

Rewrite the `only_ints` decorator s.t. it takes any number of arguments -- but only of the types you allow to be passed to the underlying function. So, if someone uses:

```python
@only_these_types(int, float)
```

Then both `int` and `float` will go through, and others won't.

In [25]:
def only_these_types(*acceptable_types):
    def middle(func):
        
        @functools.wraps(func)
        def wrapper(*args):
            
            acceptable_args = func(*[one_arg
                              for one_arg in args
                              if isinstance(one_arg, acceptable_types)])
            return acceptable_args 
        return wrapper
    return middle
    
@only_these_types(int, float)
def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total


mysum(1, 2, 'zoe', 3, 4, 'python', 5.5)

15.5

# &#x1f9f1; Stacking decorators <a class="anchor" id="stack_decs"></a>

In [26]:
# Can we stack decorators?
# Answer: yes!

@timefunc('mysum-timing.txt')
@only_these_types(int, float)
def mysum(*numbers):
    total = 0
    
    for one_number in numbers:
        total += one_number
        
    return total
# mysum = only_these_types(int, float)(mysum) i.e. mysum wrapper on only_these_types
# mysum = timefunc('filename')(mysum) i.e. mysum wrapper on timefunc

mysum(1, 2, 'zoe', 3, 4, 'python', 5.5)

15.5

In [27]:
!type mysum-timing.txt

wrapper	1726125934.9401298	0.0
mysum	1726126020.7572677	0.0
mysum	1726126071.9772425	0.0
mysum	1726126136.2596314	0.0
mysum	1726126651.30895	0.0
mysum	1726306308.5779676	0.0
mysum	1726307700.3937693	0.0
mysum	1726382543.4159172	0.0
mysum	1726382928.635957	0.0
mysum	1726388611.7492337	0.0
mysum	1726836061.3766203	0.0


How does this work?

> We have (`time_func`(`only_these_types`(`mysum`))). We decorate a function from **inside-out** &#129504;  but call it **outside-in** &#x1F4E6;  

# &#8205; &#127891; Decorating classes <a class="anchor" id="dec_class"></a>

Can we decorate classes? Yes!

Why? Because decorators are not function specific -- they work on any callable e.g. functions and classes.

> A decorator is a callable that takes a callable argument, and returns a callable value &#x1F4DE; &#x0033;&#xFE0F;&#x20E3; 

If we decorate a class, what's going to happen?

1. The outer (decorator &#x1F380;) function is invoked at defintion time. That would be when we define the class.
2. The inner (wrapper &#x1F381;) function is invoked at invocation time. That would be when we create a new instance.

In [30]:
# let's write a simple decorator which
# tells us what's happening

def louddeco(cls):
    print('Now invoking louddeco decorator')
    
    # this is invoked each time we create a new instance of our decorated class
    def wrapper(*args):
        print(f'Now invoking wrapper on {cls.__name__} with {args=}')
        new_instance = cls(*args)
        print(vars(new_instance))
        new_instance.created_at = time.time()
        return new_instance
    return wrapper

@louddeco
class MyClass:
    def __init__(self, x):
        self.x = x
    def x2(self):
        return self.x * 2
    
m1 = MyClass(10)
print(m1.x2())

m2 = MyClass(20)
print(m2.x2())

Now invoking louddeco decorator
Now invoking wrapper on MyClass with args=(10,)
{'x': 10}
20
Now invoking wrapper on MyClass with args=(20,)
{'x': 20}
40


In [31]:
vars(m1)

{'x': 10, 'created_at': 1726836158.712287}

> Use class decorators when we have common functionality across very different classes i.e. as an alternative to inheritance &#x1F468;&#x200D;&#x1F469;&#x200D;&#x1F467;&#x200D;&#x1F466;

> Class decorators can be used to add: class attributes, class methods and instance attributes &#x1F4BB;

The above example adds an attribute to each new instance of `MyClass`

```python
print(vars(new_instance))
```
Here, `x` has been defined and `__init__` run. So, any attributes we add after this line are instance attributes e.g.

```python
new_instance.created_at = time.time()
```
This allows for post-initialisation customisation &#127912;

In [32]:
# what is the type of my class?
type(MyClass)

function

In [33]:
# what is MyClass's .__name__ ?
MyClass.__name__

'wrapper'

We've gotten rid of our class in favour of a function &#128465;&#65039; But where is the original class? &#x1F50D;

Here:
```python
def louddeco(cls):
```
Our parameter `cls` is referring to it.

In [34]:
# let's do some open-heart-brain surgery
# and remove the decorator
# hint: don't do this IRL

type(m1)

__main__.MyClass

In [35]:
# remove the decorator
MyClass = type(m1)

In [36]:
m3 = MyClass(30)

In [37]:
# notice no created_at
vars(m3)

{'x': 30}

# The End!

In [38]:
from IPython.display import Image
Image(url='https://media.giphy.com/media/pZMLKQvpjq5U5MRL84/giphy.gif')